Algorithm 3.1. [y] = Chebyshev filter(x, m, a, b, a0).
Purpose: Filter x by an m degree Chebyshev polynomial which dampens on [a, b].
1. e = (b − a)/2; c = (b + a)/2;
2. σ = e/(a0 − c); σ1 = σ;
3. y = (Ax − cx)σ1/e;
4. For i = 2 : m
5.  {new} σ_new = 1 / (2/σ1−σ);
6.      ynew = 2(Ay − cy)σnew/e − σσnewx;
7.      x = y; y = ynew; σ = σnew;
8. End For

In [28]:
import numpy as np
import scipy as sp
import scipy.linalg as linalg
import numpy.matlib as ml

In [52]:
A = np.array([[1,0,0,0,0,0],
     [0,2,0,0,0,0],
     [0,0,3,0,0,0],
     [0,0,0,4,0,0],
     [0,0,0,0,5,0],
     [0,0,0,0,0,6],
    ])
x = np.array([[0.3804],
              [0.5678],
              [0.0759],
              [0.0540],
              [0.5308],
              [0.7792]
    ])

m = 4
a = 2.5
b = 5.5
a0 = 1
expected =  [ [0.3804], [0.07060463], [-0.00076315], [0.0005567], [-0.00533705], [0.09689174]]

In [53]:
def chebyshev_filter(A, x,m,a,b,a0):
    e = (b - a)/2
    c = (b + a)/2
    sigma = e/(a0 - c)
    sigma_1 = sigma;
    y = (np.dot(A,x) - c*x)*sigma_1/e
    for i in range (1,m):
        sig_new = 1 / (2/sigma_1 - sigma)
        y_new = 2*(np.dot(A,y) - c*y)*sig_new/e - sigma*sig_new*x
        x = y
        y = y_new
        sigma = sig_new;
    return y

chebyshev_filter_out = chebyshev_filter(A, x,m,a,b,a0)
print(all(abs(chebyshev_filter_out - expected) < 1e-5))

True


In [56]:
B = np.array([[1,0,0,0,0,0],[1,0,0,0,0,0],[1,0,0,0,0,0],[1,0,0,0,0,0],[1,0,0,0,0,0],[1,0,0,0,0,0]])
display(x, x.shape)
display(x.T, x.T.shape)
display(B)
display(B.T)
display(np.dot(B,x))
display(np.dot(x.T,B))

array([[0.3804],
       [0.5678],
       [0.0759],
       [0.054 ],
       [0.5308],
       [0.7792]])

(6, 1)

array([[0.3804, 0.5678, 0.0759, 0.054 , 0.5308, 0.7792]])

(1, 6)

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

array([[1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

array([[0.3804],
       [0.3804],
       [0.3804],
       [0.3804],
       [0.3804],
       [0.3804]])

array([[2.3881, 0.    , 0.    , 0.    , 0.    , 0.    ]])

In [89]:
def cheby_dav_eigh(A, x_vec, m, k_keep, dim_max, tol):
    eig_val = []
    k_c = 0
    iter_max = 100000

    x = x_vec/la.norm(x_vec)
    V = np.array([x])
    w = np.dot(A,x)
    W=np.array(w)
    
    mu = np.dot(x.T,w)[0,0]
    H=np.array([mu])
    
    r = W[:,0] - mu*x
    
    if la.norm(r) <= tol:
        eig_val[0] = mu
        k_c = 1
        H = np.array([])
    else:
        kc= 0
        
    #TODO: esimate upper bound of eigenvalues
    upperb = 10
    lowerb = (upperb+mu)/2
    a0 = lowerb
    
    k_sub = 1
    iter = 0
    while iter <= iter_max:
        iter+=1
        t = chebyshev_filter(A, x, m, lowerb, upperb, a0)
        
        #orthogonalize t against first k_sub columns of V by householder
        
        
        
        
x_vec = x
tol = 1e-12
cheby_dav_eigh(A, x_vec, m, 2, 5, tol)

In [85]:
w

array([[0.32563062],
       [0.97209815],
       [0.19491612],
       [0.18490067],
       [2.27188137],
       [4.00207226]])

In [86]:
mu

4.301453252280725

In [78]:
W[:,0]

array([0.32563062, 0.97209815, 0.19491612, 0.18490067, 2.27188137,
       4.00207226])

In [79]:
r

array([[-1.07505428, -0.42858675, -1.20576878, -1.21578423,  0.87119647,
         2.60138736],
       [-1.76508675, -1.11861922, -1.89580125, -1.9058167 ,  0.181164  ,
         1.91135489],
       [ 0.04615643,  0.69262395, -0.08455807, -0.09457353,  1.99240717,
         3.72259806],
       [ 0.12679523,  0.77326275, -0.00391927, -0.01393473,  2.07304597,
         3.80323686],
       [-1.62884768, -0.98238015, -1.75956218, -1.76957763,  0.31740307,
         2.04759396],
       [-2.5434905 , -1.89702297, -2.674205  , -2.68422045, -0.59723975,
         1.13295114]])